In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from darts.datasets import AirPassengersDataset
from darts import TimeSeries
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import data using datetime and set index to datetime
df = pd.read_csv(r'C:\Users\91783\Desktop\DataSet\1_Bhiwadi\Book_1_PM.csv',
                                parse_dates=['Date'],
                                index_col=['Date'])

df.head()

,PM2.5
Date,
2021-01-01,228.53
2021-02-01,222.37
2021-03-01,128.14
2021-04-01,68.44
2021-05-01,62.51


In [3]:
def convert2matrix(data_arr, look_back):
    X, Y =[], []
    for i in range(len(data_arr)-look_back):
        d=i+look_back  
        X.append(data_arr[i:d,])
        Y.append(data_arr[d,])
        return np.array(X), np.array(Y)

In [4]:
#Split data set into testing dataset and train dataset
train_size = 300
train, test =df.values[0:train_size,:],df.values[train_size:len(df.values),:]
# setup look_back window 
look_back = 30
#convert dataset into right shape in order to input into the DNN
trainX, trainY = convert2matrix(train, look_back)
testX, testY = convert2matrix(test, look_back)

In [ ]:
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
def model_dnn(look_back):
    model_1=Sequential()
    model_1.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model_1.add(Dense(8, activation='relu'))
    model_1.add(Dense(1))
    model_1.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model_1

In [ ]:
model_1=model_dnn(look_back)


In [ ]:
history=model_1.fit(trainX,trainY, epochs=15, batch_size=8, verbose=2, validation_data=(testX,testY),callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
#Taking predictions
Y_pred1 = model_1.predict(testX)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(testY, Y_pred1)

print(mse)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
#Taking predictions
Y_pred1 = model_1.predict(testX)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(testY, Y_pred1)

print(mse)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
Y_pred1

In [ ]:
testY

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, EarlyStopping
def model_rnn(look_back):
    model_2=Sequential()
    model_2.add(SimpleRNN(units=32, input_shape=(1,look_back), activation="relu"))
    model_2.add(Dense(8, activation='relu'))
    model_2.add(Dense(1))
    model_2.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model_2

In [ ]:
model_2=model_rnn(look_back)

In [ ]:
history_2=model_2.fit(trainX,trainY, epochs=15, batch_size=8, verbose=2, validation_data=(testX,testY),callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)


In [ ]:
#Taking predictions
Y_pred2 = model_2.predict(testX)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(testY, Y_pred2)

print(mse)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
#Normalising data
from sklearn.preprocessing import MinMaxScaler

values = df.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_dataset = scaler.fit_transform(values)

In [ ]:
scaled_dataset[:2,:]

In [ ]:
#Creating a window for previous data
def to_supervised(train):
    window_size = 4
    X = []
    Y = []
    for i in range(window_size, len(train)):
        X.append(train[i-window_size:i,:])
        Y.append(train[i,0:1])
    
    return X,Y

In [ ]:
X, Y = to_supervised(scaled_dataset)
X = np.array(X)
Y = np.array(Y)
print('Y' ,Y.shape)
print('X' ,X.shape)

In [ ]:
#Splitting the dataset
n_train = 3*30
X_train, X_test = X[n_train:,] , X[:n_train,]
print('X_train' ,X_train.shape)
print('X_test' ,X_test.shape)

Y_train, Y_test = Y[n_train:,] , Y[:n_train,]
print('Y_train' ,Y_train.shape)
print('Y_test' ,Y_test.shape)

In [ ]:
#Importing LSTM model
from keras.models import Sequential
from keras.layers import Dense, Dropout,LSTM
model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')   

In [ ]:
history = model.fit(X_train, Y_train, epochs=50,
                    batch_size=8, validation_data=(X_test, Y_test),
                    verbose=2, shuffle=False)

In [ ]:
#Taking predictions
Y_pred_3 = model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, Y_pred_3)

print(mse)
rmse = np.sqrt(mse)
print(rmse)

In [ ]:
print(Y_pred1)


In [ ]:
print(Y_pred_3)

In [ ]:
# Finding the split point index to keep 1 month (30 days) data as test set
n_test = 61
train_data = df.iloc[:len(df) - n_test]
test_data = df.iloc[len(df) - n_test:]

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA,ARIMAResults
model = ARIMA(train_data['PM2.5'],order=(1,2,3))
results = model.fit()
results.summary()

In [ ]:
# Obtain predicted values on the test set
start = len(train_data)
end = len(train_data) + len(test_data) - 1
predictions = results.predict(start=start,end=end,dynamic=True,typ='levels').rename('ARIMA Predictions')

In [ ]:
predictions

In [ ]:
# Compare predictions to expected values
for i in range(len(predictions)):
    print(f"predicted = {predictions[i]}, expected = {test_data['PM2.5'][i]}")

In [ ]:
compare = pd.DataFrame(columns=['Actual','Predicted'])
compare['Actual'] = test_data['PM2.5']
compare['Predicted'] = predictions
compare.tail()

In [ ]:
d1=predictions

In [ ]:
d1

In [ ]:
d=test_data['PM2.5']

In [ ]:
d.info()

In [ ]:
d3=d.append([d1])

In [ ]:
d4=pd.DataFrame(d3)

In [ ]:
d4

In [ ]:
Y_pred_3